In [1]:
#! pip install lightgbm catboost

In [2]:
#!install scikit-learn

In [3]:
#! pip install openpyxl

In [4]:
#! pip install gdown

In [5]:
#! pip install tensorflow

In [6]:
#! pip install xgboost

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
import gdown
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping



/Users/ericacarneiro/Code/Github/BIP_Course/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [8]:
# File ID and destination file name
file_id = "1SKHN7CcEMoam_npoPghXtZyEAd8JP_Fj"
url = f"https://drive.google.com/uc?id={file_id}"

# Download the file
gdown.download(url, "trainSet.xlsx", quiet=False)

# Open the file with pandas
trainSet = pd.read_excel("trainSet.xlsx", engine="openpyxl")
#print(trainSet.head())

Downloading...
From (original): https://drive.google.com/uc?id=1SKHN7CcEMoam_npoPghXtZyEAd8JP_Fj
From (redirected): https://drive.google.com/uc?id=1SKHN7CcEMoam_npoPghXtZyEAd8JP_Fj&confirm=t&uuid=141a3de7-bb18-4efc-9aab-bf6bc2301305
To: /Users/ericacarneiro/Code/Github/BIP_Course/trainSet.xlsx
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50.5M/50.5M [00:14<00:00, 3.37MB/s]


In [9]:
read_file = pd.read_excel ("trainSet.xlsx")

In [10]:
read_file.to_csv ("trainSet.csv", index = None, header=True) 
                
# reading csv file and converting into a dataframe 
df = pd.DataFrame(pd.read_csv("trainSet.csv")) 
                    
# show the dataframe 
df.head()

/var/folders/g9/q6km00k56ln8jq2fc_5z8gr40000gn/T/ipykernel_12685/255366979.py:4: DtypeWarning: Columns (16,24,26,36,38,64,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.DataFrame(pd.read_csv("trainSet.csv"))


,RPAKREP_VEHICLE_HKEY,COMPANY,OFFICE,OFFICE_MAIN_BRAND,CHASSIS_NUMBER,MANUFACTURER_SHORT,MANUFACTURER,VEHICLE_GROUP,VEHICLE_TYPE,MODEL_CODE,...,CUSTOMER_SALE_GROUP_NAME,CUSTOMER_SALE_GROUP2,CUSTOMER_SALE_GROUP2_NAME,SCALED_CURRENT_VALUE,SCALED_INVENTURAL_VALUE,SCALED_REPORT_VALUE,SCALED_VALUATION_PRICE,SCALED_GUIDE_PRICE,SCALED_TOTAL_SALES_PRICE_BASIS,SCALED_TOTAL_SALE_PRICE
0,77c1af48604226d47b2ccf7be44df009,44.0,TA,TOY,WF01XXERK1JR43779,FOR,Ford,ECS,ECOSPORT 1.0 Titanium Eco,NaN,...,EV privat,EVPR,Endverbraucher private,0.052907,0.000000,0.950043,0.0,0.002975,0.011018,0.011321
1,39587e4f9324333232d2ffaa2f314e6d,12.0,44,VOL,YV1XZK7V5S2487523,VOL,Volvo,XC4,"XC40 B3 B DKG Core 120 kW, 5-t",536K7170F1,...,NaN,NaN,NaN,0.061605,0.000000,0.950043,0.0,0.006136,0.026411,0.019844
2,a91470879cf756368c1d0099a2fa939a,27.0,33,SKO,WF0DXXSK0RX009819,FOR,Ford,TOC,Tourneo Connect 2.0 EcoBl,26,...,EV privat,EVPR,Endverbraucher private,0.063286,0.000000,0.950043,0.0,0.001378,0.022102,0.022053
3,61363a68ff967e7f0c4b80b46a55b141,33.0,K1,V,WVWZZZAWZPU075160,V,Volkswagen,POL,"POLO LIFE 1,0 L 59 KW (80 PS)",AE13GV,...,Interne Kunden,INT,Interne Kunden,0.042565,0.029537,0.897257,0.0,0.001378,0.000000,0.000572
4,fd5b09f7d857904d03a36d1a479f2ca4,10.0,96,FOR,WF0PXXGCHPRC20073,FOR,Ford,FOC,FOCUS TITA 5W 1.0T 125 MHEV M6,15,...,NaN,NaN,NaN,0.054914,0.000000,0.950043,0.0,0.004863,0.019715,0.013360


In [11]:
# Features and target
X = df.drop(columns=["LAID_UP_TIME"])
y = df["LAID_UP_TIME"]

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Identifying categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object", "category"]).columns
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

In [13]:
print(preprocessor)

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 Index(['COMPANY', 'MILEAGE', 'OPERATING_HOURS', 'MILAGE_IN_FIELD',
       'MILAGE_SALES', 'OPERATING_HOURS_SALES', 'CERTIFICATE_TYPE_DATE',
       'FRONT_TIRES_CONDITION', 'REAR_TIRES_CONDITION', 'NUMBER_DOORS',
       'NUMBER_SEATS', 'PERMITTED_TOTAL_WEIGHT', 'MAX_TRAILOR_LOAD',
       'CURB_WEIGHT', 'YEAR_CONSTRUCTION', 'CONSTRUCTION...
       'PURCHASE_BOOKING_DATE', 'PRICE_LIST', 'DAY_OF_REGISTRATION',
       'SOLD_CUSTOMER_ID', 'SOLD_INVOICE_COSTUMER_ID',
       'SOLD_INVOICE_COSTUMER_ID2', 'CUSTOMER_TYPE', 'CUSTOMER_GROUP',
       'CUSTOMER_GROUP_NAME', 'CUSTOMER_FEATURE', 'CUSTOMER_FEATURE_NAME',
       'SALE_CUSTOMER_ID2', 'CUSTOMER_SALE_GROUP', 'CUSTOMER_SALE_GROUP_NAME',
       'CUSTOMER_SALE_GROUP2', 'CUSTOMER_SALE_GROUP2_NAME'],
      dtype='object'))])


In [14]:
# Identifying columns with mixed types
mixed_type_columns = [col for col in X.columns if X[col].apply(type).nunique() > 1]
print("Columns with mixed types:", mixed_type_columns)

Columns with mixed types: ['OFFICE', 'OFFICE_MAIN_BRAND', 'CHASSIS_NUMBER', 'MANUFACTURER_SHORT', 'MANUFACTURER', 'VEHICLE_GROUP', 'VEHICLE_TYPE', 'MODEL_CODE', 'VARIANT', 'RIM_KEY', 'COLOR_CODE', 'COLOR_CODE_NAME', 'COLOR', 'COLOR_TYPE', 'UPHOLSTERY_CODE', 'UPHOLSTERY', 'UPHOLSTERY_CODE_ALT', 'CERTIFICATE_TYPE', 'FACTORY_NUMBER', 'ENGINE_ID', 'ENGINE_TYPE', 'ENGINE_ID_ALT', 'TRANSMISSION', 'TRANSMISSION_TYPE', 'TRANSMISSION_ID', 'TRANSMISSION_SHORT', 'TRANSMISSION_NAME', 'RIMS', 'FRONT_TIRES', 'REAR_TIRES', 'ACCIDENT_VEHICLE', 'COMMISSION_NUMBER', 'LEASING_CONTRACT_DATE', 'LEASING_START', 'LEASING_END', 'FINANCING_TYPE', 'FINANCING_TYPE_NAME', 'KAT_VEHICLE', 'FUEL_TYPE', 'FUEL_TYPE_NAME', 'DRIVE_TYPE', 'DRIVE_TYPE_NAME', 'VEHICLE_MODEL_ID_NAME', 'COMMISSION_TYPE', 'COMMISSION_TYPE_NAME', 'DEMONSTRATION_STATUS', 'PURCHASE_DATE', 'PURCHASE_BOOKING_DATE', 'PRICE_LIST', 'DAY_OF_REGISTRATION', 'SOLD_CUSTOMER_ID', 'SOLD_INVOICE_COSTUMER_ID', 'SOLD_INVOICE_COSTUMER_ID2', 'CUSTOMER_TYPE', 'CU

In [15]:
for col in mixed_type_columns:
    X[col] = X[col].astype(str)

In [16]:
#Updating preprocessor after identifying and fixing mix type columns due to
# error output on preprocessing data on NN

In [17]:
## Appying Gradient Boost

In [18]:
# Gradient Boosting Model
xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)


In [19]:
# Creating Neural Network Model
def create_nn(input_dim):
    model = Sequential()
    model.add(Dense(64, activation="relu", input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation="linear"))
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mse")
    return model

In [20]:
# investingating issue
for col in mixed_type_columns:
    print(f"Unique values in {col}: {X[col].unique()}")

Unique values in OFFICE: ['TA' '44' '33' 'K1' '96' '13' '62' '46' '59' '60' '39' 'G4' 'K5' 'K3'
 '23' '20' '89' 'K2' '22' '52' 'P1' '31' '71' 'P5' '25' 'T7' 'J9' 'H5'
 'P4' '21' '10' '85' 'K4' '67' '64' 'G3' 'T1' '70' 'P6' 'K8' '61' '54'
 '49' '75' '95' '56' 'G2' '15' '5' 'J7' 'P2' '80' '28' 'T2' '18' '11' '38'
 '93' '92' 'T9' '76' '84' 'KA' 'S9' '69' '45' '58' '2' 'K7' '43' '55' '17'
 'J2' '30' '57' 'J8' '19' '37' '16' '65' '86' '94' 'T3' '1' 'nan' '14'
 '72' 'TE' 'TD' '6' '36' 'H9' 'K6' 'T4' '4' 'KC' '47' '87' 'K9' '81' 'P3'
 'J5' '91' '68' 'G1' '12' 'FD' 'KD' '26' '77' '27' '24']
Unique values in OFFICE_MAIN_BRAND: ['TOY' 'VOL' 'SKO' 'V' 'FOR' 'FIA' 'nan' 'WDB' 'OPE' 'BMW' 'C' 'JAG' 'LEX'
 'MAZ' 'FER' 'AM']
Unique values in CHASSIS_NUMBER: ['WF01XXERK1JR43779' 'YV1XZK7V5S2487523' 'WF0DXXSK0RX009819' ...
 'WV4ZZZT12PS001454' 'WF0EXXTTREPL57038' 'ZARPAHDNXM7D09853']
Unique values in MANUFACTURER_SHORT: ['FOR' 'VOL' 'V' 'HYU' 'LAR' 'OPE' 'FIA' 'S' 'MG' 'JEE' 'BMW' 'WDB' 'ORA'
 'MIT' 'P

In [21]:
# dropping nan

In [22]:
nan_columns = X.columns[X.isna().any()].tolist()
print("Columns with NaN values:", nan_columns)

Columns with NaN values: ['COMPANY', 'MILEAGE', 'OPERATING_HOURS', 'MILAGE_IN_FIELD', 'MILAGE_SALES', 'OPERATING_HOURS_SALES', 'CERTIFICATE_TYPE_DATE', 'FRONT_TIRES_CONDITION', 'REAR_TIRES_CONDITION', 'NUMBER_DOORS', 'NUMBER_SEATS', 'PERMITTED_TOTAL_WEIGHT', 'MAX_TRAILOR_LOAD', 'CURB_WEIGHT', 'YEAR_CONSTRUCTION', 'CONSTRUCTION_MONTH', 'NUMBER_AXLE', 'NUMBER_ENGINE_CYLINDER', 'REPAIR_RKZ', 'OPTICAL_CONDITION', 'TECHNICAL_CONDITION', 'HORSEPOWER', 'KW', 'CCM', 'NUMBER_OWNERS', 'IS_USED_CAR', 'LEASING_MILAGE', 'PAINT_TYPE', 'VEHICLE_MODEL_ID', 'PURCHASE_MILAGE', 'PURCHASE_OPERATION_HOURS', 'AT_LOCATION_SINCE', 'MILAGE_SALE', 'OPERATION_HOURS_SALE', 'SCALED_CURRENT_VALUE', 'SCALED_INVENTURAL_VALUE', 'SCALED_REPORT_VALUE', 'SCALED_VALUATION_PRICE', 'SCALED_GUIDE_PRICE', 'SCALED_TOTAL_SALES_PRICE_BASIS', 'SCALED_TOTAL_SALE_PRICE']


In [23]:
for col in categorical_cols:
    X[col] = X[col].fillna("missing")
    nan_columns = X.columns[X.isna().any()]
print(f"Columns with NaN values:\n{nan_columns}")

Columns with NaN values:
Index(['COMPANY', 'MILEAGE', 'OPERATING_HOURS', 'MILAGE_IN_FIELD',
       'MILAGE_SALES', 'OPERATING_HOURS_SALES', 'CERTIFICATE_TYPE_DATE',
       'FRONT_TIRES_CONDITION', 'REAR_TIRES_CONDITION', 'NUMBER_DOORS',
       'NUMBER_SEATS', 'PERMITTED_TOTAL_WEIGHT', 'MAX_TRAILOR_LOAD',
       'CURB_WEIGHT', 'YEAR_CONSTRUCTION', 'CONSTRUCTION_MONTH', 'NUMBER_AXLE',
       'NUMBER_ENGINE_CYLINDER', 'REPAIR_RKZ', 'OPTICAL_CONDITION',
       'TECHNICAL_CONDITION', 'HORSEPOWER', 'KW', 'CCM', 'NUMBER_OWNERS',
       'IS_USED_CAR', 'LEASING_MILAGE', 'PAINT_TYPE', 'VEHICLE_MODEL_ID',
       'PURCHASE_MILAGE', 'PURCHASE_OPERATION_HOURS', 'AT_LOCATION_SINCE',
       'MILAGE_SALE', 'OPERATION_HOURS_SALE', 'SCALED_CURRENT_VALUE',
       'SCALED_INVENTURAL_VALUE', 'SCALED_REPORT_VALUE',
       'SCALED_VALUATION_PRICE', 'SCALED_GUIDE_PRICE',
       'SCALED_TOTAL_SALES_PRICE_BASIS', 'SCALED_TOTAL_SALE_PRICE'],
      dtype='object')


In [24]:
X[numerical_cols] = X[numerical_cols].fillna(X[numerical_cols].mean())


In [25]:
X[categorical_cols] = X[categorical_cols].fillna("missing")


In [26]:
print("Remaining NaN values:", X.isna().sum().sum())


Remaining NaN values: 99071


In [27]:
nan_columns = X.columns[X.isna().any()]
print(f"Columns with remaining NaN values:\n{nan_columns}")


Columns with remaining NaN values:
Index(['PAINT_TYPE'], dtype='object')


In [28]:
print(X[nan_columns].isna().sum())


PAINT_TYPE    99071
dtype: int64


In [29]:
X[nan_columns] = X[nan_columns].fillna(X[nan_columns].mean())


In [30]:
X[nan_columns] = X[nan_columns].fillna("missing")


In [31]:
print("Remaining NaN values:", X.isna().sum().sum())


Remaining NaN values: 0


In [32]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [33]:
print(X['PAINT_TYPE'].isna().sum(), "out of", len(X['PAINT_TYPE']))


0 out of 99071


In [34]:
print(X.columns)

Index(['RPAKREP_VEHICLE_HKEY', 'COMPANY', 'OFFICE', 'OFFICE_MAIN_BRAND',
       'CHASSIS_NUMBER', 'MANUFACTURER_SHORT', 'MANUFACTURER', 'VEHICLE_GROUP',
       'VEHICLE_TYPE', 'MODEL_CODE',
       ...
       'CUSTOMER_SALE_GROUP_NAME', 'CUSTOMER_SALE_GROUP2',
       'CUSTOMER_SALE_GROUP2_NAME', 'SCALED_CURRENT_VALUE',
       'SCALED_INVENTURAL_VALUE', 'SCALED_REPORT_VALUE',
       'SCALED_VALUATION_PRICE', 'SCALED_GUIDE_PRICE',
       'SCALED_TOTAL_SALES_PRICE_BASIS', 'SCALED_TOTAL_SALE_PRICE'],
      dtype='object', length=105)


In [35]:
mixed_type_columns = [col for col in X.columns if X[col].apply(type).nunique() > 1]
print(f"Columns with mixed types: {mixed_type_columns}")


Columns with mixed types: []


In [36]:
for col in categorical_cols:
    print(f"Column: {col}")
    print(X[col].unique())

Column: RPAKREP_VEHICLE_HKEY
['77c1af48604226d47b2ccf7be44df009' '39587e4f9324333232d2ffaa2f314e6d'
 'a91470879cf756368c1d0099a2fa939a' ... '3728dc4cbef2dbf42e840c45f0eb08c6'
 '4dee3ce14a1942b1c70fe9ad2862f42f' 'ad711cff51def6c12ba88870ebd6345c']
Column: OFFICE
['TA' '44' '33' 'K1' '96' '13' '62' '46' '59' '60' '39' 'G4' 'K5' 'K3'
 '23' '20' '89' 'K2' '22' '52' 'P1' '31' '71' 'P5' '25' 'T7' 'J9' 'H5'
 'P4' '21' '10' '85' 'K4' '67' '64' 'G3' 'T1' '70' 'P6' 'K8' '61' '54'
 '49' '75' '95' '56' 'G2' '15' '5' 'J7' 'P2' '80' '28' 'T2' '18' '11' '38'
 '93' '92' 'T9' '76' '84' 'KA' 'S9' '69' '45' '58' '2' 'K7' '43' '55' '17'
 'J2' '30' '57' 'J8' '19' '37' '16' '65' '86' '94' 'T3' '1' 'nan' '14'
 '72' 'TE' 'TD' '6' '36' 'H9' 'K6' 'T4' '4' 'KC' '47' '87' 'K9' '81' 'P3'
 'J5' '91' '68' 'G1' '12' 'FD' 'KD' '26' '77' '27' '24']
Column: OFFICE_MAIN_BRAND
['TOY' 'VOL' 'SKO' 'V' 'FOR' 'FIA' 'nan' 'WDB' 'OPE' 'BMW' 'C' 'JAG' 'LEX'
 'MAZ' 'FER' 'AM']
Column: CHASSIS_NUMBER
['WF01XXERK1JR43779' 'YV1XZK7V

In [37]:
X[categorical_cols] = X[categorical_cols].fillna("missing")


In [ ]:
for col in categorical_cols:
    print(f"Column: {col}")
    print(X[col].unique())



In [45]:
print(X[X.isna().any(axis=1)])  # Rows with any NaN values


Empty DataFrame
Columns: [RPAKREP_VEHICLE_HKEY, COMPANY, OFFICE, OFFICE_MAIN_BRAND, CHASSIS_NUMBER, MANUFACTURER_SHORT, MANUFACTURER, VEHICLE_GROUP, VEHICLE_TYPE, MODEL_CODE, VARIANT, MILEAGE, OPERATING_HOURS, MILAGE_IN_FIELD, MILAGE_SALES, OPERATING_HOURS_SALES, RIM_KEY, COLOR_CODE, COLOR_CODE_NAME, COLOR, COLOR_TYPE, UPHOLSTERY_CODE, UPHOLSTERY, UPHOLSTERY_CODE_ALT, CERTIFICATE_TYPE, CERTIFICATE_TYPE_DATE, FACTORY_NUMBER, ENGINE_ID, ENGINE_TYPE, ENGINE_ID_ALT, TRANSMISSION, TRANSMISSION_TYPE, TRANSMISSION_ID, TRANSMISSION_SHORT, TRANSMISSION_NAME, RIMS, FRONT_TIRES, FRONT_TIRES_CONDITION, REAR_TIRES, REAR_TIRES_CONDITION, NUMBER_DOORS, NUMBER_SEATS, PERMITTED_TOTAL_WEIGHT, MAX_TRAILOR_LOAD, CURB_WEIGHT, YEAR_CONSTRUCTION, CONSTRUCTION_MONTH, NUMBER_AXLE, NUMBER_ENGINE_CYLINDER, REPAIR_RKZ, OPTICAL_CONDITION, TECHNICAL_CONDITION, ACCIDENT_VEHICLE, COMMISSION_NUMBER, HORSEPOWER, KW, CCM, NUMBER_OWNERS, IS_USED_CAR, LEASING_CONTRACT_DATE, LEASING_START, LEASING_END, LEASING_MILAGE, FINA

In [44]:
# Removing PAINT_TYPE column because it has no useful information for the model
print(X.columns)
if 'PAINT_TYPE' in X.columns:
    X = X.drop(columns=['PAINT_TYPE'])
print("Categorical columns:", categorical_cols)


Index(['RPAKREP_VEHICLE_HKEY', 'COMPANY', 'OFFICE', 'OFFICE_MAIN_BRAND',
       'CHASSIS_NUMBER', 'MANUFACTURER_SHORT', 'MANUFACTURER', 'VEHICLE_GROUP',
       'VEHICLE_TYPE', 'MODEL_CODE',
       ...
       'CUSTOMER_SALE_GROUP_NAME', 'CUSTOMER_SALE_GROUP2',
       'CUSTOMER_SALE_GROUP2_NAME', 'SCALED_CURRENT_VALUE',
       'SCALED_INVENTURAL_VALUE', 'SCALED_REPORT_VALUE',
       'SCALED_VALUATION_PRICE', 'SCALED_GUIDE_PRICE',
       'SCALED_TOTAL_SALES_PRICE_BASIS', 'SCALED_TOTAL_SALE_PRICE'],
      dtype='object', length=104)
Categorical columns: Index(['RPAKREP_VEHICLE_HKEY', 'OFFICE', 'OFFICE_MAIN_BRAND', 'CHASSIS_NUMBER',
       'MANUFACTURER_SHORT', 'MANUFACTURER', 'VEHICLE_GROUP', 'VEHICLE_TYPE',
       'MODEL_CODE', 'VARIANT', 'RIM_KEY', 'COLOR_CODE', 'COLOR_CODE_NAME',
       'COLOR', 'COLOR_TYPE', 'UPHOLSTERY_CODE', 'UPHOLSTERY',
       'UPHOLSTERY_CODE_ALT', 'CERTIFICATE_TYPE', 'FACTORY_NUMBER',
       'ENGINE_ID', 'ENGINE_TYPE', 'ENGINE_ID_ALT', 'TRANSMISSION',
       'TRA

In [46]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="mean")),
            ("scaler", StandardScaler())
        ]), numerical_cols),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
            ("encoder", OneHotEncoder(handle_unknown="ignore"))
        ]), categorical_cols),
    ]
)

In [47]:
numerical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

X_num_test = X_train[numerical_cols]
X_num_preprocessed = numerical_pipeline.fit_transform(X_num_test)
print("Numerical pipeline test successful.")

Numerical pipeline test successful.


/Users/ericacarneiro/Code/Github/BIP_Course/venv/lib/python3.9/site-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['PAINT_TYPE']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


In [50]:
X_train[categorical_cols] = X_train[categorical_cols].fillna("missing")


In [51]:
X_train[categorical_cols] = X_train[categorical_cols].astype(str)


In [ ]:
print(X_train[categorical_cols].dtypes)  # Should show all columns as "object"
for col in categorical_cols:
    print(f"Unique values in {col}: {X_train[col].unique()}")


In [53]:
categorical_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="missing")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

X_cat_test = X_train[categorical_cols]
X_cat_preprocessed = categorical_pipeline.fit_transform(X_cat_test)
print("Categorical pipeline test successful.")


Categorical pipeline test successful.


In [55]:
# Combining both, numerical and categorical, preprocessed features
X_train_preprocessed = np.hstack([X_num_preprocessed, X_cat_preprocessed])
print("Combined preprocessed features shape:", X_train_preprocessed.shape)


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [56]:
print("X_num_preprocessed shape:", X_num_preprocessed.shape)
print("X_cat_preprocessed shape:", X_cat_preprocessed.shape)


X_num_preprocessed shape: (79256, 40)
X_cat_preprocessed shape: (79256, 406063)


In [57]:
for col in categorical_cols:
    print(f"Column: {col}, Unique categories: {X_train[col].nunique()}")


Column: RPAKREP_VEHICLE_HKEY, Unique categories: 70727
Column: OFFICE, Unique categories: 111
Column: OFFICE_MAIN_BRAND, Unique categories: 16
Column: CHASSIS_NUMBER, Unique categories: 69878
Column: MANUFACTURER_SHORT, Unique categories: 76
Column: MANUFACTURER, Unique categories: 94
Column: VEHICLE_GROUP, Unique categories: 595
Column: VEHICLE_TYPE, Unique categories: 12148
Column: MODEL_CODE, Unique categories: 4499
Column: VARIANT, Unique categories: 183
Column: RIM_KEY, Unique categories: 15
Column: COLOR_CODE, Unique categories: 1453
Column: COLOR_CODE_NAME, Unique categories: 8
Column: COLOR, Unique categories: 4117
Column: COLOR_TYPE, Unique categories: 8
Column: UPHOLSTERY_CODE, Unique categories: 1502
Column: UPHOLSTERY, Unique categories: 4060
Column: UPHOLSTERY_CODE_ALT, Unique categories: 8
Column: CERTIFICATE_TYPE, Unique categories: 9
Column: FACTORY_NUMBER, Unique categories: 105
Column: ENGINE_ID, Unique categories: 18124
Column: ENGINE_TYPE, Unique categories: 1141
Co

In [61]:
# Temporarily adding y_train to X_train
X_train_temp = X_train.copy()
X_train_temp["LAID_UP_TIME"] = y_train

# Performing target encoding
for col in categorical_cols:
    target_encoding = X_train_temp.groupby(col)["LAID_UP_TIME"].mean()
    X_train[col] = X_train[col].map(target_encoding)

# Removing the temporary target column
del X_train_temp



In [62]:
print(X_train[categorical_cols].head())


       RPAKREP_VEHICLE_HKEY      OFFICE  OFFICE_MAIN_BRAND  CHASSIS_NUMBER  \
60971             87.333333   76.825397          73.085997       87.333333   
92056            100.000000  109.202658          97.929828      100.000000   
48961             14.000000   62.851459          71.369334       14.000000   
38453            127.000000  103.139117          97.929828      127.000000   
22189             43.000000   55.581590          55.581590       43.000000   

       MANUFACTURER_SHORT  MANUFACTURER  VEHICLE_GROUP  VEHICLE_TYPE  \
60971           73.349949     72.958997      49.407809     26.676923   
92056          100.825175    102.673913     104.302343    158.500000   
48961           72.513387     72.513387      68.993311     69.463087   
38453           93.519440     93.519440     106.782783    128.764331   
22189           66.588639     66.588639      52.371429     37.872549   

       MODEL_CODE     VARIANT  ...  CUSTOMER_TYPE  CUSTOMER_GROUP  \
60971   93.616431   88.132878

In [63]:
for col in categorical_cols:
    X_train[col] = X_train[col].fillna(y_train.mean())


In [64]:
print("Remaining NaN values:", X_train.isna().sum().sum())


Remaining NaN values: 165090


In [65]:
X_train = X_train.fillna(y_train.mean())


In [66]:
# Combining numerical and target-encoded categorical data
X_train_combined = np.hstack([X_num_preprocessed, X_train.values])
print("Shape of combined data:", X_train_combined.shape)


Shape of combined data: (79256, 145)


In [67]:
from sklearn.model_selection import train_test_split

X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    X_train_combined, y_train, test_size=0.2, random_state=42
)


In [68]:
def create_nn(input_dim):
    model = Sequential()
    model.add(Dense(64, activation="relu", input_dim=input_dim))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation="linear"))  # Output for regression
    model.compile(optimizer=Adam(learning_rate=0.001), loss="mse", metrics=["mse"])
    return model

nn_model = create_nn(X_train_final.shape[1])

/Users/ericacarneiro/Code/Github/BIP_Course/venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [69]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True)

history = nn_model.fit(
    X_train_final, y_train_final,
    validation_data=(X_val_final, y_val_final),
    epochs=100,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


Epoch 1/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 2s 647us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 591us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 634us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 598us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 592us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 592us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 590us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 598us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/100
1982/1982 ━━━━━━━━━━━━━━━━━━━━ 1s 588us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
E

In [71]:
print("NaN in X_train_final:", np.isnan(X_train_final).sum())
print("NaN in X_val_final:", np.isnan(X_val_final).sum())

print("Inf in X_train_final:", np.isinf(X_train_final).sum())
print("Inf in X_val_final:", np.isinf(X_val_final).sum())


NaN in X_train_final: 0
NaN in X_val_final: 0
Inf in X_train_final: 0
Inf in X_val_final: 0


In [72]:
print("NaN in y_train_final:", np.isnan(y_train_final).sum())
print("NaN in y_val_final:", np.isnan(y_val_final).sum())

print("Inf in y_train_final:", np.isinf(y_train_final).sum())
print("Inf in y_val_final:", np.isinf(y_val_final).sum())


NaN in y_train_final: 39
NaN in y_val_final: 10
Inf in y_train_final: 0
Inf in y_val_final: 0


In [73]:
y_train_final = np.nan_to_num(y_train_final)
y_val_final = np.nan_to_num(y_val_final)


In [77]:
from sklearn.metrics import mean_squared_error
import numpy as np

y_val_pred = nn_model.predict(X_val_final)
rmse = np.sqrt(mean_squared_error(y_val_final, y_val_pred))
print(f"Validation RMSE: {rmse}")


496/496 ━━━━━━━━━━━━━━━━━━━━ 0s 262us/step


ValueError: Input contains NaN.